# 麻雀点数表

本稿では麻雀点数表を NumPy を使って作成する。単なる Jupyter Notebook の練習用文書だ。

以下のことを考慮するのは後回しにしよう：

* 存在しない符・飜の組み合わせに対応する要素をマスクする（ただしこれは是非行いたい）
* 点数表の眺めを良くする（これは当面本稿の目的からやや外れているため）

## 材料を用意する

何はさておき NumPy をインポートする。
それから次のデータおよび機能を用意する：

* 符（配列）
* 飜（配列）
* 十の位で数を切り上げる（関数）

In [1]:
import numpy as np
np.set_printoptions(linewidth=160)

### 符

符を 20から 110 まで 10 刻みで配列 `FU` に格納する。ただし七対子用に 25 も格納する。

In [2]:
from itertools import chain

FU = np.asarray(list(chain((20, 25), range(30, 120, 10))))
FU

array([ 20,  25,  30,  40,  50,  60,  70,  80,  90, 100, 110])

### 飜

飜を 1 から 12 まで 1 刻みで配列 `HAN` に格納する。必要ならもっと大きい飜を追加してもよい。

In [3]:
HAN = np.arange(1, 14)
HAN

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

それに加えあらかじめ `HAN` の各値 $n$ について $2^n$ を計算した飜の配列 `HAN2` も用意する。一飜のときに 1 になるように指数を調整する：

In [4]:
HAN2 = 1 << (HAN - 1)
HAN2

array([   1,    2,    4,    8,   16,   32,   64,  128,  256,  512, 1024, 2048, 4096], dtype=int32)

### 十の位で数を切り上げる

最初に整数を引数に取り、整数を返す関数 `roundup` を定義する。
それから `np.ndarray` を引数に取る同様の関数 `roundupv` を定義する。

In [5]:
def roundup(score):
    """Round `score` up to the nearest 100."""

    remainder = score % 100
    if remainder:
        score += 100
        score -= remainder

    return score

roundupv = np.vectorize(roundup)

### 基本点数表を用意する

ここが本稿のミソなのだが、関数 `np.meshgrid` を利用して符配列と飜配列の直積のような `np.ndarray` オブジェクトを生成し、それを基に最終的な点数表を得るための基礎オブジェクトを生成する。これにより多重配列の各軸を走査するようなループを書くという、危険かつ煩雑な作業を回避する。

In [6]:
hanm, fum = np.meshgrid(HAN2, FU)
basic_points = hanm * fum * 8

ここでオブジェクト `basic_points` を見ると次のようになっている。これを適当な定数で乗じて十の位を切り上げれば、子・親それぞれのロンアガリの点数表になるはずだ。

In [7]:
basic_points

array([[    160,     320,     640,    1280,    2560,    5120,   10240,   20480,   40960,   81920,  163840,  327680,  655360],
       [    200,     400,     800,    1600,    3200,    6400,   12800,   25600,   51200,  102400,  204800,  409600,  819200],
       [    240,     480,     960,    1920,    3840,    7680,   15360,   30720,   61440,  122880,  245760,  491520,  983040],
       [    320,     640,    1280,    2560,    5120,   10240,   20480,   40960,   81920,  163840,  327680,  655360, 1310720],
       [    400,     800,    1600,    3200,    6400,   12800,   25600,   51200,  102400,  204800,  409600,  819200, 1638400],
       [    480,     960,    1920,    3840,    7680,   15360,   30720,   61440,  122880,  245760,  491520,  983040, 1966080],
       [    560,    1120,    2240,    4480,    8960,   17920,   35840,   71680,  143360,  286720,  573440, 1146880, 2293760],
       [    640,    1280,    2560,    5120,   10240,   20480,   40960,   81920,  163840,  327680,  655360, 1310720, 26

## 子の点数表

子がツモアガリするときに他の子が支払う点数を計算する（ただし「青天井」な要素はそのままである）：

In [8]:
roundupv(basic_points)

array([[    200,     400,     700,    1300,    2600,    5200,   10300,   20500,   41000,   82000,  163900,  327700,  655400],
       [    200,     400,     800,    1600,    3200,    6400,   12800,   25600,   51200,  102400,  204800,  409600,  819200],
       [    300,     500,    1000,    2000,    3900,    7700,   15400,   30800,   61500,  122900,  245800,  491600,  983100],
       [    400,     700,    1300,    2600,    5200,   10300,   20500,   41000,   82000,  163900,  327700,  655400, 1310800],
       [    400,     800,    1600,    3200,    6400,   12800,   25600,   51200,  102400,  204800,  409600,  819200, 1638400],
       [    500,    1000,    2000,    3900,    7700,   15400,   30800,   61500,  122900,  245800,  491600,  983100, 1966100],
       [    600,    1200,    2300,    4500,    9000,   18000,   35900,   71700,  143400,  286800,  573500, 1146900, 2293800],
       [    700,    1300,    2600,    5200,   10300,   20500,   41000,   82000,  163900,  327700,  655400, 1310800, 26

## 親の点数表

親がツモアガリするときに子から得る点棒の点数（ホニャララオールのホニャララ）を計算する（ただし「青天井」な要素はそのままである）：

In [9]:
roundupv(basic_points * 2)

array([[    400,     700,    1300,    2600,    5200,   10300,   20500,   41000,   82000,  163900,  327700,  655400, 1310800],
       [    400,     800,    1600,    3200,    6400,   12800,   25600,   51200,  102400,  204800,  409600,  819200, 1638400],
       [    500,    1000,    2000,    3900,    7700,   15400,   30800,   61500,  122900,  245800,  491600,  983100, 1966100],
       [    700,    1300,    2600,    5200,   10300,   20500,   41000,   82000,  163900,  327700,  655400, 1310800, 2621500],
       [    800,    1600,    3200,    6400,   12800,   25600,   51200,  102400,  204800,  409600,  819200, 1638400, 3276800],
       [   1000,    2000,    3900,    7700,   15400,   30800,   61500,  122900,  245800,  491600,  983100, 1966100, 3932200],
       [   1200,    2300,    4500,    9000,   18000,   35900,   71700,  143400,  286800,  573500, 1146900, 2293800, 4587600],
       [   1300,    2600,    5200,   10300,   20500,   41000,   82000,  163900,  327700,  655400, 1310800, 2621500, 52

## 基本点表を麻雀に近づける

本節では満貫以上の符・飜の組み合わせに対応するアガリの点数を青天井ではなく、標準的な麻雀ルールのそれに修正する。

### 基本点を超える要素を修正する
NumPy 配列オブジェクト `basic_points` の要素のうち、値が 2000 を超えるものを一律 2000 に置換する。説明用に `basic_points` のコピー `W` に対して操作をする：

In [10]:
W = basic_points.copy()
W[W > 2000] = 2000
W

array([[ 160,  320,  640, 1280, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 200,  400,  800, 1600, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 240,  480,  960, 1920, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 320,  640, 1280, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 400,  800, 1600, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 480,  960, 1920, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 560, 1120, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 640, 1280, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 720, 1440, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 800, 1600, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000],
       [ 880, 1760, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000]])

### 満貫以上のアガリの点を修正する
NumPy 配列オブジェクト `basic_points` の 5 飜以上の要素が修正候補になる。
そのアルゴリズムは次のようになる：

* 5 飜以下かつ基本点が 2000 のものを満貫とする。これは前節で実施済み。
* 6 以上は 8 飜未満ならば跳満とし、基本点を 3000 とする。
* 8 以上 11 未満ならば倍満とし、基本点を 4000 とする。
* 11 以上 13 未満ならば三倍満とし、基本点を 6000 とする。
* 13 飜以上は数え役満とし、基本点を 8000 とする。

これをオブジェクト `W` に適用するには、NumPy のたいへん便利な配列要素操作機能、slicing を利用する。ここでは添字は飜数そのものではなく、飜数を格納する配列 `HAN` の添字であることに注意（つまり 1 ズレる）：

In [11]:
W[:,5:7] = 3000
W[:,7:10] = 4000
W[:,10:12] = 6000
W[:,12:] = 8000
W

array([[ 160,  320,  640, 1280, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 200,  400,  800, 1600, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 240,  480,  960, 1920, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 320,  640, 1280, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 400,  800, 1600, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 480,  960, 1920, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 560, 1120, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 640, 1280, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 720, 1440, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 800, 1600, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 880, 1760, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000]])

最後に十の位を切り上げれば、基本点数表（子ツモによる子の支払い）が確定する：

In [12]:
roundupv(W)

array([[ 200,  400,  700, 1300, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 200,  400,  800, 1600, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 300,  500, 1000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 400,  700, 1300, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 400,  800, 1600, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 500, 1000, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 600, 1200, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 700, 1300, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 800, 1500, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 800, 1600, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000],
       [ 900, 1800, 2000, 2000, 2000, 3000, 3000, 4000, 4000, 4000, 6000, 6000, 8000]])

## 雑感
* 麻雀を知るプログラマーならば、エクセルでも何でも構わないから一度は麻雀点数表を自作するべきだろう。青天井ルールの麻雀を開催する事態になったときに落ち着いて対処できる。

* NumPy の配列オブジェクトを操作する以上は、手で `for` ループを書くのは可能な限り避けるべきだろう。専門家からの未熟の謗りは免れまい。

* 関数 `np.fromfunc` を `np.meshgrid` の代わりに利用するのもよい。実質的には同じことになる。また、引数の関数の内容に符や飜で条件分岐するような処理を含められないことに注意すること。

* 繰り上げ関数を自作したのが気に入らない。NumPy には `ceil` や `round_` というものがあるが、今回の目的に適合させるのがやや面倒なので仕方がない。

## 参考文献

* [NumPy User Guide](http://docs.scipy.org/doc/numpy/user/)
* [麻雀の得点計算](https://ja.wikipedia.org/wiki/%E9%BA%BB%E9%9B%80%E3%81%AE%E5%BE%97%E7%82%B9%E8%A8%88%E7%AE%97#.E5.AD.90.E3.81.AE.E7.82.B9.E6.95.B0.E6.97.A9.E8.A6.8B.E8.A1.A8)
